In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing

import tensorflow as tf
from tensorflow import keras
import datetime

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import zipfile

train_zip = '/kaggle/input/aerial-cactus-identification/train.zip'
with zipfile.ZipFile(train_zip, 'r') as zip_ref:
    zip_ref.extractall('/kaggle/tmp/')

test_zip = '/kaggle/input/aerial-cactus-identification/test.zip'
with zipfile.ZipFile(test_zip, 'r') as zip_ref:
    zip_ref.extractall('/kaggle/tmp/')
    
for dirname, _, filenames in os.walk('/kaggle/tmp'):
    for filename in filenames[:10]:
        print(os.path.join(dirname, filename))    

In [ ]:
class Cnn_Model:

    #-------------------------------------------
    #
    #-------------------------------------------
    def __init__(self):
        self.BATCH_SZ = 32
        self.LR_RATE = 0.0001
        self.EPOCHS = 150
        self.MODEL_CKP_PATH = '/kaggle/tmp/model_ckpoint'
        self.MODEL_TRAIN_DATA = '/kaggle/tmp/train'
        self.MODEL_TEST_DATA = '/kaggle/tmp/test'
        self.model = None
        
    #-------------------------------------------
    #
    #-------------------------------------------
    def get_model(self):
        return self.model
            
    #-------------------------------------------
    #
    #-------------------------------------------
    def load_dataframe(self):
        self.df = pd.read_csv('../input/aerial-cactus-identification/train.csv')
        self.df.has_cactus = self.df.has_cactus.astype(str)
        
    #-------------------------------------------
    #
    #-------------------------------------------
    def make_train_gen(self):

        self.train_gen = keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255,
            zoom_range=0.2,
            width_shift_range=0.4,
            height_shift_range=0.4,
            horizontal_flip=True,
            vertical_flip=True,
            rotation_range=60,
            brightness_range=[0.8,1.1],
        )

        self.train_direcIter = self.train_gen.flow_from_dataframe(
            dataframe=self.df,
            directory=self.MODEL_TRAIN_DATA,
            x_col='id',
            y_col='has_cactus',
            target_size=(32,32),
            batch_size=self.BATCH_SZ,
            shuffle=True,
            class_mode='binary'
        )

    #-------------------------------------------
    #
    #-------------------------------------------
    def build_model(self):

        self.model = keras.models.Sequential([
            keras.layers.Conv2D(64, (3, 3), input_shape=(32, 32, 3)),
            keras.layers.MaxPooling2D(pool_size=(2, 2)),
            keras.layers.Dropout(0.2),
            keras.layers.Conv2D(64, (3, 3)),
            keras.layers.MaxPooling2D(pool_size=(2, 2)),
            keras.layers.Dropout(0.1),
            keras.layers.Flatten(),
            keras.layers.Dense(128, activation="relu"),
            keras.layers.Dropout(0.3),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dropout(0.2),            
            keras.layers.Dense(32, activation="relu"),
            keras.layers.Dropout(0.1),
            keras.layers.Dense(1, activation="sigmoid")
        ])

        self.model.summary()

    #-------------------------------------------
    #
    #-------------------------------------------
    def compile_model(self):

        optz = keras.optimizers.Adam(learning_rate=self.LR_RATE)

        self.model.compile(
            optimizer=optz,
            loss=keras.losses.binary_crossentropy,
            metrics=["acc"]
        )

    #-------------------------------------------
    #
    #-------------------------------------------
    def train_model(self):

        def scheduler(epoch, lr):
            return lr * tf.math.exp(-0.01)

        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=self.MODEL_CKP_PATH,
            save_weights_only=False,
            monitor='acc',
            mode='max',
            save_best_only=True,
        )

        log_dir = "logs\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

        tensorboard_callback = tf.keras.callbacks.TensorBoard(
            log_dir=log_dir,
            histogram_freq=1,
            update_freq="epoch",
            profile_batch=0,
            embeddings_freq=0
        )

        callback_list = [
            keras.callbacks.LearningRateScheduler(scheduler),
#             model_checkpoint_callback,
#             tensorboard_callback,
        ]

        self.model.fit(
            x=self.train_direcIter,
            epochs=self.EPOCHS,
            callbacks=callback_list,
            steps_per_epoch=(self.train_direcIter.samples//self.BATCH_SZ),
        )

    #-------------------------------------------
    #
    #-------------------------------------------
    def load_model(self):
        self.model = tf.keras.models.load_model(self.MODEL_CKP_PATH)
        print("> Model load done...")

    #-------------------------------------------
    #
    #-------------------------------------------
    def predict_values(self):
        
        for file_name in os.listdir(self.MODEL_TEST_DATA)[:10]:
            fullpath_img = os.path.join(self.MODEL_TEST_DATA, file_name)
            image = tf.keras.preprocessing.image.load_img(fullpath_img)
            input_arr = keras.preprocessing.image.img_to_array(image)
            input_arr = input_arr/255
            print(type(input_arr))
            print(input_arr.shape)
            input_arr = np.array([input_arr])  # Convert single image to a batch.
            predictions = self.model.predict(input_arr)
            print(predictions)
        
    #-------------------------------------------
    #
    #-------------------------------------------
    def evaluate_model(self, steps=100):

        result = self.model.evaluate(self.train_direcIter, steps=steps)
        print(result)        
        

In [ ]:
#-------------------------------------------
#
#-------------------------------------------
cnn_obj = Cnn_Model()
cnn_obj.load_dataframe()
cnn_obj.make_train_gen()

cnn_obj.build_model()
cnn_obj.compile_model()
cnn_obj.train_model()

In [ ]:
# cnn_obj = Cnn_Model()
# cnn_obj.load_model()
model = cnn_obj.get_model()

In [ ]:
test_gen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

test_files = os.listdir('/kaggle/tmp/test')
df_test = pd.DataFrame(test_files, columns=['id'])

test_direcIter = test_gen.flow_from_dataframe(
    dataframe=df_test,
    directory='/kaggle/tmp/test',
    x_col='id',
    y_col=None,
    target_size=(32,32),
    batch_size=len(test_files),
    class_mode=None
)

test_direcIter.reset()
y_pred = model.predict(test_direcIter, steps=1)
df_test['has_cactus'] = np.array(y_pred).flatten()
df_test.to_csv('submission.csv', index=False)